In [1]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from os import listdir
from os.path import isfile, join
import fnmatch
import os

"""
This module is used to scrap data from eSight website. It takes as input a table extracted from the 'Mass Edit Meters' tab and return
a CSV file with each meter details. When a line does not have any Site.MeterCode it means that the tool did not manage to extracted 
any information. 

Call the start_scrap(input_dataframe) to start the scraping process

Requirements: firefox + selenium library
"""
  
def prepare_dataset(data):
    """
    Split the dataset "Path" into individual cell
    """
    for i in range(0, 5, 1):
        data['Name'+str(i+1)] = data.apply(split_path, axis=1, args=(i,))
    return

def split_path(row, index): 
    """
    Function that split the path into pieces and return the one at the given index
    """
    string_chain = str(row['Path']).split(' \\ ')
    if len(string_chain)>index:
        return string_chain[index]
    
def clean_dataset(data):
    data.drop_duplicates(inplace=True)#remove duplicates
    data.sort_values(['Name1', 'Name2', 'Name3', 'Name4', 'Name5'],inplace=True)#sort the meter list by their location ("path")
    data.reset_index(drop=True, inplace=True) #reinitialize the index
    return

def merge_dataframe(startwith, mypath):
    
    result = pd.DataFrame()
    
    if mypath=="" or mypath is None:
        mypath=os.getcwd()
        
    frames = []
    for file in listdir(mypath):
        
        if fnmatch.fnmatch(file, startwith+'*.csv'):
            data = pd.read_csv(file, encoding="iso-8859-1")
            frames.append(data)
            
    result = pd.concat(frames)       
    result.reset_index(drop=True, inplace=True)#make sure that the index starts at 0
    result.to_csv("merged_file.csv", encoding="utf-8")
    return result

In [40]:
def data_scraping(data):
    
    data.reset_index(drop=True, inplace=True)#make sure that the index starts at 0
    
    df = pd.DataFrame(columns=['Name','Site.MeterCode','Meter Prefix','Meter Number', 'Cost Contract', 'Revenue Contract' , 'Meter Type', 'Meter Code',
                               'Meter', 'Type','Import Code', 'No of register', 'Read Frequency', 'Correction Factor', 'Serial Number', 'Cumulative', 
                               'Billing', 'Main Meter', 'Pro-Rata', 'Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Path'], index=data.index.values)
    
    driver = webdriver.Firefox()
    driver.get(#ADD URL PAGE)

    ## Login

    #Get the webelement for the username and password
    username = driver.find_element_by_id("txtUsername")
    password = driver.find_element_by_id("txtPassword")

    #Fill in the login details and click the login button
    username.send_keys(#ADD USERNAME HERE)
    password.send_keys(#ADD PASSWORD HERE)
    driver.find_element_by_id('btnLogin').click()

    #Go to the page we are interested in
    driver.get(#ADD URL PAGE)
    
    #initialize variables
    contract = ""
    site = ""
    meter = ""
    name1=""
    name2=""
    name3=""
    name4=""
    name5=""
    
    try:
        
        for numRow, line in data.iterrows():
            print("Start:",numRow, line['Name1'], line['Name2'], line['Name3'], line['Name4'], line['Name5'], "meter:", line['Meter'])
            
            split_path = [x for x in line.filter(regex=("Name.*")) if not x is None]
            temp_name = split_path[-1] #Store the last branch of the path
            
            for attempt in range(3):
                try:
                    if line['Name1'] !=  name1:
                        print("branch1", line['Name1'])
                        if name1!="":
                            branch1.click()
                        branch1= driver.find_element_by_xpath("//span[text()='"+str(line['Name1'])+"']/preceding-sibling::span[@class='rtPlus']")
                        branch1.click()
                        time.sleep(2)

                    if (line['Name2'] != name2) and (not line['Name2'] is None):
                        if name2 != "" and line['Name1']==name1:
                            name1=line['Name1']
                            print("Close branch 2 ", name1, name2)
#                             element = WebDriverWait(branch1, 10).until(EC.visibility_of_element_located((By.XPATH, 
#                             "//span[text()='"+str(name1)+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(name2)+"']/preceding-sibling::span[@class='rtPlus']"))) #Wait and check for the visibility of the webelement
                            branch2= branch1.find_element_by_xpath("//span[text()='"+str(name1)+"']/ancestor::li[1]/ul/li/div/span[text()='"+
                                                                    str(name2)+"']/preceding-sibling::span[@class='rtMinus']"
                                                                  )#get the webelement
                            branch2.click()
                        
                        print("branch2", line['Name2'])
                        element = WebDriverWait(branch1, 10).until(EC.visibility_of_element_located((By.XPATH, 
                            "//span[text()='"+str(line['Name1'])+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(line['Name2'])+"']/preceding-sibling::span[@class='rtPlus']"))) #Wait and check for the visibility of the webelement
                        branch2= branch1.find_element_by_xpath(
                            "//span[text()='"+str(line['Name1'])+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(line['Name2'])+"']/preceding-sibling::span[@class='rtPlus']"
                                                )#get the webelement
                        driver.execute_script("return arguments[0].scrollIntoView();", branch2) #Scroll so the webelement is in the view
                        branch2.click()
                       

                    if (line['Name3'] != name3) and (not line['Name3'] is None):
                        print("branch3", line['Name3'])
                        
                        element = WebDriverWait(branch1, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[text()='"+str(line['Name2'])+"']/ancestor::li[1]/ul/li/div[starts-with(@class, 'rt')]/span[text()='"+str(line['Name3'])+"']/preceding-sibling::span[@class='rtPlus']"))) #Wait and check for the visibility of the webelement
                        branch2= branch2.find_element_by_xpath("//span[text()='"
                                                                +str(line['Name2'])+"']/ancestor::li[1]/ul/li/div[starts-with(@class, 'rt')]/span[text()='"
                                                                +str(line['Name3'])+"']/preceding-sibling::span[@class='rtPlus']"
                                                              )
            
                        driver.execute_script("return arguments[0].scrollIntoView();", branch2)
                        branch2.click()
                        
                    if (line['Name4'] != name4) and (not line['Name4'] is None):
                        print("branch4", line['Name4'])
                        element = WebDriverWait(branch1, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[text()='"+str(line['Name3'])+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(line['Name4'])+"']/preceding-sibling::span[@class='rtPlus']"))) #Wait and check for the visibility of the webelement
                        branch2= branch2.find_element_by_xpath("//span[text()='"+str(line['Name3'])+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(line['Name4'])+"']/preceding-sibling::span[@class='rtPlus']"
                                                                )#get the webelement
                        branch2.click()

                    if (line['Name5'] != name5) and (not line['Name5'] is None):
                        print("branch5", line['Name5'])
                        element = WebDriverWait(branch1, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[text()='"+str(line['Name4'])+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(line['Name5'])+"']/preceding-sibling::span[@class='rtPlus']"))) #Wait and check for the visibility of the webelement
                        branch2= branch2.find_element_by_xpath("//span[text()='"+str(line['Name4'])+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(line['Name5'])+"']/preceding-sibling::span[@class='rtPlus']"
                                                              )#get the webelement
                        branch2.click()

                    if  line['Meter'] != "":   #If a meter exists, we look for it

                        element = WebDriverWait(branch2, 15).until(
                            EC.visibility_of_element_located((By.XPATH, "//span[text()='"+str(temp_name)+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(line['Meter'])+"']"))
                        )
    #                     branch3= branch2.find_element_by_xpath("//span[text()='"+str(line['Meter'])+"']")
                        branch3= branch2.find_element_by_xpath("//span[text()='"+str(temp_name)+"']/ancestor::li[1]/ul/li/div/span[text()='"+str(line['Meter'])+"']")
                        branch3.click()

                        name = "empty"
                        Site_MeterCode = "empty"
                        Meter_Prefix = "empty"
                        Meter_Type = "empty"
                        Meter_Number = "empty"
                        Type = "empty"
                        Import_Code = "empty"
                        Read_Frequency = "empty"
                        Correction_Factor = "empty"
                        Serial_Number = "empty"
                        Revenue_Contract = "empty"
                        Cumulative ="empty"
                        Billing ="empty"
                        MainMeter ="empty"
                        ProRata ="empty"
                        Cost_contract="empty"
                        Revenue_contract="empty"
                        No_Registers="empty"

                        time.sleep(1)

                        element = WebDriverWait(driver, 15).until(
                            EC.visibility_of_element_located((By.XPATH, "//table/tbody/tr[contains(@class, 'meter')]"))
                        )

                        #go through the table that store the meter details and extract the relevant information 
                        for row in driver.find_elements_by_xpath("//table/tbody/tr[contains(@class, 'meter')]"):

                            cell = row.find_elements_by_tag_name("td")
                            for index, key in enumerate(cell):
                                meterdetails = str(key.text)
#                                 print(index, meterdetails)
                                if  meterdetails == "Name":
                                    name = str(cell[index+1].text)
                    #                             print("name:", name)
                                elif  meterdetails =="Site.MeterCode":
                                    Site_MeterCode = str(cell[index+1].text)
                    #                             print("Sitemetercode:", Site_MeterCode)
                                elif  meterdetails == "Meter Prefix":
                                    Meter_Prefix = str(cell[index+1].text)
                    #                             print("meterPrefix:", Meter_Prefix)
                                elif  meterdetails == "Meter Type":
                                    Meter_Type = str(cell[index+1].text)
                    #                             print("Meter Type:", Meter_Type)
                                elif  meterdetails =="Meter Number":
                                    Meter_Number = str(cell[index+1].text)
                    #                             print("Meter Number", Meter_Number)
                                elif  meterdetails =="Type":
                                    Type = str(cell[index+1].text)
                    #                             print("Type", Type)
                                elif  meterdetails =="Import Code":
                                    Import_Code = str(cell[index+1].text)
                    #                             print("Import Code", Import_Code)
                                elif  meterdetails =="Read Frequency":
                    #                             print("Read Frequency", cell[index+1].text)
                                    Read_Frequency = str(cell[index+1].text)
                                elif  meterdetails =="Correction Factor":
                                    Correction_Factor = str(cell[index+1].text)
                                elif  meterdetails =="Cost Contract":
                                    Cost_contract = str(cell[index+1].text)    
                                elif  meterdetails =="Revenue Contract":
                                    Revenue_contract = str(cell[index+1].text)      
                                elif  meterdetails =="No of Registers":
                                    No_Registers = str(cell[index+1].text)    
                                elif  meterdetails =="Serial Number":
                                    Serial_Number = str(cell[index+1].text)
                                elif  meterdetails =="Cumulative":
                                    img = cell[index+1].find_element_by_tag_name("img")
                                    if img.get_attribute("src") == "http://energy.bbworkplace.com/Images/tick.png":
                                        Cumulative="Yes"
                                    else:
                                        Cumulative="No"
                                elif  meterdetails =="Billing":
                                    img = cell[index+1].find_element_by_tag_name("img")
                                    if img.get_attribute("src") == "http://energy.bbworkplace.com/Images/tick.png":
                                        Billing="Yes"
                                    else:
                                        Billing="No"
                                elif  meterdetails =="Main Meter":
                                    img = cell[index+1].find_element_by_tag_name("img")
                                    if img.get_attribute("src") == "http://energy.bbworkplace.com/Images/tick.png":
                                        MainMeter="Yes"
                                    else:
                                        MainMeter="No"
                                elif  meterdetails =="Pro-Rata":
                                    img = cell[index+1].find_element_by_tag_name("img")
                                    if img.get_attribute("src") == "http://energy.bbworkplace.com/Images/tick.png":
                                        ProRata="Yes"
                                    else:
                                        ProRata="No"      
                                        

                        #Store the information within the dataframe

                        df.iloc[numRow, :]= pd.Series({'Name':name,'Site_MeterCode':Site_MeterCode,'Meter Prefix':Meter_Prefix, 'Meter Type':Meter_Type, 
                                                       'Meter Code': line['Meter Code'], 'Meter': line['Meter'],'Meter Number':Meter_Number,'Type':Type,
                                                       'Import Code':Import_Code, 'Read Frequency':Read_Frequency, 'Correction Factor':Correction_Factor, 
                                                       'Serial Number':Serial_Number, 'Cumulative':Cumulative, 'Billing':Billing, 'Main Meter':MainMeter, 
                                                       'No of register':No_Registers,'Cost Contract':Cost_contract, 'Revenue_contract':Revenue_contract,
                                                       'Pro-Rata':ProRata, 'Name1':line['Name1'],'Name2':line['Name2'], 'Name3':line['Name3'], 
                                                       'Name4':line['Name4'], 'Name5':line['Name5'], 'Path':line['Path']})

                        meter = line['Meter']
                        name1=line['Name1']
                        name2=line['Name2']
                        name3=line['Name3']
                        name4=line['Name4']
                        name5=line['Name5']
                        print("End ",numRow, line['Name1'], line['Name2'], line['Name3'], line['Name4'], line['Name5'])
                except:
                    print("wait a bit more...", attempt)
                    time.sleep(5)
                else:

                    break 
                                       
    finally: #Failed all the attempt
        print("Finally:", numRow)
        driver.quit()         
                  
    return df
                    
def start_scrap(meter_list):
#     meter_list = pd.read_csv("test-scraping - min.csv", encoding='utf-8', dtype={'Meter':str, 'Meter Code':str}) #import csv file
    
    prepare_dataset(meter_list)
    clean_dataset(meter_list)
    
    step = 200
    number_meter = meter_list.shape[0]
    number_of_iteration = number_meter//step
    
    for x in range(0,number_of_iteration+1, 1):
        high_threshold=step+x*step
        low_threshold=0+x*step
        if high_threshold>number_meter:
            high_threshold=number_meter            
        print(low_threshold, high_threshold)
        result = data_scraping(meter_list.iloc[low_threshold:high_threshold, :].copy())
        result.to_csv("Mass meter export "+str(x)+".csv")
    
    return merge_dataframe("Mass meter export", "")


# meter_list = pd.read_csv("test-scraping - min.csv", encoding='utf-8', dtype={'Meter':str, 'Meter Code':str}) #import csv file
# result = start_scrap(meter_list)

0 7
Start: 0 Bassetlaw Schools (BAS) Calculated Meters None None None meter: Bassetlaw Gas
branch1 Bassetlaw Schools (BAS)
branch2 Calculated Meters
End  0 Bassetlaw Schools (BAS) Calculated Meters None None None
Start: 1 Bassetlaw Schools (BAS) Calculated Meters None None None meter: Bassetlaw Electricity
End  1 Bassetlaw Schools (BAS) Calculated Meters None None None
Start: 2 Bassetlaw Schools (BAS) Calculated Meters None None None meter: Bassetlaw Water
End  2 Bassetlaw Schools (BAS) Calculated Meters None None None
Start: 3 Bassetlaw Schools (BAS) Elizabethan Academy None None None meter: 1160000176730 AMR
Close branch 2  Bassetlaw Schools (BAS) Calculated Meters
branch2 Elizabethan Academy
End  3 Bassetlaw Schools (BAS) Elizabethan Academy None None None
Start: 4 Bassetlaw Schools (BAS) Elizabethan Academy None None None meter: Electricity Core Hours
End  4 Bassetlaw Schools (BAS) Elizabethan Academy None None None
Start: 5 Bassetlaw Schools (BAS) Elizabethan Academy None None Non

1160000000000.0
  Mass Edit                                           Path  \
0     False    Bassetlaw Schools (BAS) \ Calculated Meters   
1     False    Bassetlaw Schools (BAS) \ Calculated Meters   
2     False    Bassetlaw Schools (BAS) \ Calculated Meters   
3     False  Bassetlaw Schools (BAS) \ Elizabethan Academy   
4     False  Bassetlaw Schools (BAS) \ Elizabethan Academy   
5     False  Bassetlaw Schools (BAS) \ Elizabethan Academy   
6     False  Bassetlaw Schools (BAS) \ Elizabethan Academy   

                      Meter      Meter Code Import Code Unnamed: 5  
0             Bassetlaw Gas          BASGas                   Edit  
1     Bassetlaw Electricity  BASElectricity                   Edit  
2           Bassetlaw Water        BASWater                   Edit  
3         1160000176730 AMR        1.16E+12    1.16E+12       Edit  
4    Electricity Core Hours           ECore                   Edit  
5  Electricity Out of Hours          EOther                   Edit  
6   